In [ ]:
import os
import pyspark
conf = pyspark.SparkConf()
conf.set("spark.ui.proxyBase", "/user/" + os.environ["JUPYTERHUB_USER"] + "/proxy/4041")
conf.set("spark.driver.memory", "16g")

sc = pyspark.SparkContext(conf = conf)
spark = pyspark.SQLContext.getOrCreate(sc)

In [2]:
import re
import statistics
import json
import numpy as np
import textdistance as td
import pyspark.sql.functions as F
from pyspark.sql.functions import udf, col, rank, collect_set, explode, array
from pyspark.sql.window import Window
import pyspark.sql.types as types

# **Dataset**

Download the dataset from here - https://www.kaggle.com/datasets/ravindrasinghrana/job-description-dataset/data and place the unzipped `.csv` in the root directory.

Keep in mind this dataset has been synthetically generated so the aggregates and insights we extract are very unrealistic.

Example:
| Job Id | Experience | Qualifications | Salary Range | location | Country | latitude | longitude | Work Type | Company Size | Job Posting Date | Preference | Contact Person | Contact | Job Title | Role | Job Portal | Job Description | Benefits | skills | Responsibilities | Company | Company Profile |
| - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - | - |
| 398454096642776 | 2 to 12 Years | BCA | $56K-$116K | Ashgabat | Turkmenistan | 38.9697 | 59.5563 | Intern | 100340 | 2022-12-19 | Female | Francisco Larsen | 461-509-4216 | Web Developer | Frontend Web Developer | Idealist | Frontend Web Developers design and implement user interfaces for websites, ensuring they are visuall... | {'Health Insurance, Retirement Plans, Paid Time Off (PTO), Flexible Work Arrangements, Employee Assi... | HTML, CSS, JavaScript Frontend frameworks (e.g., React, Angular) User experience (UX) | Design and code user interfaces for websites, ensuring a seamless and visually appealing user experi... | PNC Financial Services Group | {"Sector":"Financial Services","Industry":"Commercial Banks","City":"Pittsburgh","State":"Pennsylvan... |

In [3]:
jobs = spark.read.option("header", True).option("inferSchema", True).csv("../job_descriptions.csv")
jobs.count()

1615940

In [4]:
jobs.groupBy("Role").count().sort("Role").toPandas()

,Role,count
0,API Developer,3483
1,Accessibility Developer,3513
2,Account Executive,7063
3,Account Manager,3474
4,Account Strategist,3460
...,...,...
371,Wedding Consultant,3492
372,Wedding Coordinator,3552
373,Wedding Designer,3353
374,Wedding Planner,6902


# **Feature 1**

Comparison of job to other jobs of the same position.

Example: I am looking for a new "software engineer" position.  When I look at a job posting, I'd like to see how this posting compares to other "software engineer" positions.  For example, "this posting has a lower salary than other postings of similar positions, and requires more qualifications.

Input:
```json
{
    "role": "job role"
    "salary": 100000,
    "education": "BS",
    "experience": 5,
    "num_skills": 5,
    "num_responsibilities": 5,
    "num_benefits": 5
}
```

Output:
```json
{
    "role": "Frontend Web Developer",
    "salary": "less than average",
    "education": "less than average",
    "experience": "less than average",
    "skills": "greater than average",
    "responsibilities": "greater than average",
    "benefits": "greater than average",
}
```

In [142]:
role_stats = {}
roles = jobs.groupBy("Role").count().sort("Role").toPandas()

for i, row in roles.iterrows():
    role_stats[row["Role"]] = {}

In [143]:
def get_avg_salary(salary_str):
    range = [int(n) for n in re.findall(r"\d+", salary_str)]
    return statistics.mean(range) * 1000

udf_avg_salary = udf(get_avg_salary, types.FloatType())

salaries = jobs.withColumn("Salary", udf_avg_salary(col("Salary Range")))\
.groupBy("Role").mean("Salary").sort("Role").toPandas()

for i, row in salaries.iterrows():
    role_stats[row["Role"]]["salary"] = row["avg(Salary)"]

salaries

,Role,avg(Salary)
0,API Developer,82636.602452
1,Accessibility Developer,82385.600455
2,Account Executive,82449.157303
3,Account Manager,82809.915014
4,Account Strategist,82440.628638
...,...,...
371,Wedding Consultant,82349.743882
372,Wedding Coordinator,82566.101695
373,Wedding Designer,82452.353283
374,Wedding Planner,82525.977817


In [144]:
def get_avg_ed(ed_str):
    # Bachelor's
    if ed_str[0] == "B":
        return 1
    # Master's
    if ed_str[0] == "M":
        return 2
    # Doctorate
    if ed_str[0] == "P":
        return 3
    else:
        return 0

udf_avg_ed = udf(get_avg_ed, types.IntegerType())

educations = jobs.withColumn("Education", udf_avg_ed(col("Qualifications")))\
.groupBy("Role").mean("Education").sort("Role").toPandas()

for i, row in educations.iterrows():
    role_stats[row["Role"]]["education"] = row["avg(Education)"]

educations

,Role,avg(Education)
0,API Developer,1.614126
1,Accessibility Developer,1.597780
2,Account Executive,1.600878
3,Account Manager,1.585492
4,Account Strategist,1.600578
...,...,...
371,Wedding Consultant,1.615979
372,Wedding Coordinator,1.606700
373,Wedding Designer,1.590218
374,Wedding Planner,1.601710


In [145]:
def get_avg_exp(exp_str):
    range = [int(n) for n in re.findall(r"\d+", exp_str)]
    return statistics.mean(range)

udf_avg_exp = udf(get_avg_exp, types.FloatType())

exps = jobs.withColumn("Years Experience", udf_avg_exp(col("experience")))\
.groupBy("Role").mean("Years Experience").sort("Role").toPandas()

for i, row in exps.iterrows():
    role_stats[row["Role"]]["experience"] = row["avg(Years Experience)"]

exps

,Role,avg(Years Experience)
0,API Developer,7.003116
1,Accessibility Developer,7.026810
2,Account Executive,6.961190
3,Account Manager,6.964162
4,Account Strategist,7.009029
...,...,...
371,Wedding Consultant,6.962441
372,Wedding Coordinator,6.971982
373,Wedding Designer,7.065859
374,Wedding Planner,7.018487


In [146]:
def get_avg_skills(skills_str):
    split = re.split(r"(?=\s[A-Z])", skills_str)
    skills = [s.strip() for s in split if s.strip()]
    return len(skills)

udf_avg_skills = udf(get_avg_skills, types.IntegerType())

skills = jobs.withColumn("num_skills", udf_avg_skills(col("skills")))\
.groupBy("Role").mean("num_skills").sort("Role").toPandas()

for i, row in skills.iterrows():
    role_stats[row["Role"]]["num_skills"] = row["avg(num_skills)"]

skills

,Role,avg(num_skills)
0,API Developer,5.0
1,Accessibility Developer,14.0
2,Account Executive,7.0
3,Account Manager,5.0
4,Account Strategist,6.0
...,...,...
371,Wedding Consultant,3.0
372,Wedding Coordinator,12.0
373,Wedding Designer,3.0
374,Wedding Planner,5.0


In [147]:
def get_avg_res(responsibilities_str):
    responsibilities = [r.strip() for r in responsibilities_str.split(".") if r.strip()]
    return len(responsibilities)

udf_avg_res = udf(get_avg_res, types.IntegerType())

responsibilities = jobs.withColumn("num_responsibilities", udf_avg_res(col("responsibilities")))\
.groupBy("Role").mean("num_responsibilities").sort("Role").toPandas()

for i, row in responsibilities.iterrows():
    role_stats[row["Role"]]["num_responsibilities"] = row["avg(num_responsibilities)"]

responsibilities

,Role,avg(num_responsibilities)
0,API Developer,3.0
1,Accessibility Developer,3.0
2,Account Executive,3.0
3,Account Manager,3.0
4,Account Strategist,3.0
...,...,...
371,Wedding Consultant,3.0
372,Wedding Coordinator,3.0
373,Wedding Designer,3.0
374,Wedding Planner,3.0


In [148]:
def get_avg_ben(benefits_str):
    benefits = [b.strip() for b in benefits_str[2:-2].split(",") if b.strip()]
    return len(benefits)

udf_avg_ben = udf(get_avg_ben, types.IntegerType())

benefits = jobs.withColumn("num_benefits", udf_avg_ben(col("benefits")))\
.groupBy("Role").mean("num_benefits").sort("Role").toPandas()

for i, row in benefits.iterrows():
    role_stats[row["Role"]]["num_benefits"] = row["avg(num_benefits)"]

benefits

,Role,avg(num_benefits)
0,API Developer,5.0
1,Accessibility Developer,5.0
2,Account Executive,5.0
3,Account Manager,5.0
4,Account Strategist,5.0
...,...,...
371,Wedding Consultant,5.0
372,Wedding Coordinator,5.0
373,Wedding Designer,5.0
374,Wedding Planner,5.0


In [149]:
with open("../outputs/role_stats.json", "w") as f:
    json.dump(role_stats, f)

## Demonstration

In [3]:
with open("../outputs/role_stats.json") as f:
    role_stats = json.load(f)

In [ ]:
# using Google Gemini
import google.generativeai as genai
import numbers

genai.configure(api_key = "")
model = genai.GenerativeModel("gemini-1.0-pro")

def compare_job_posting(job_description, model, role_stats):
    output = {}
    for key in  ["salary", "education", "experience", "num_skills", "num_responsibilities", "num_benefits", "education"]:
        output[key] = None
        
    format = '''
    {
     "role": "job role"
     "salary": 100000,
     "education": "BS",
     "experience": 5,
     "num_skills": 5,
     "num_responsibilities": 5,
     "num_benefits": 5
    }
    '''
    description = ("where role is the job position or title, "
                   "salary is the estimated salary in dollars, "
                   "ed is the required education, "
                   "exp is the desired years of experience, "
                   "num_skills is number of skills listed, "
                   "num_responsibilities is the number of responsibilities listed, "
                   "and num_benefits is the number of benefits listed.  "
                   "If something is not provided, put a 'null' instead.  "
                   "Do not include any delimiters or special characters "
                   "other than what is needed to load this JSON string into Python.")
    
    is_json = False
    attempts = 0
    while not is_json:
        prompt = "Based off this job description:\n" + job_description + \
        "\nCan you give me a JSON object in this format with this information:\n" + format + description
        response = model.generate_content(prompt)

        # make sure we don't query too many times
        attempts += 1
        if attempts >= 3:
            return output
            
        # keep looping until we get a valid JSON format
        try:
            parsed_info = json.loads(re.findall(r"\{.*?\}", response.text)[0])
            role = parsed_info["role"]
            is_json = True
        except:
            pass

    prompt = "Here is a list of job roles: \n" + str(list(role_stats.keys())) + f"\nCan you give me only one role from that list which is most similar to '{parsed_info['role']}'?"
    response = model.generate_content(prompt)

    # every so often AI will return a role that's not actually in the list
    # if this is the case, find the most similar role in the list to the returned role using cosine similarity
    ratios = []
    for role in list(role_stats.keys()):
        ratios.append(td.cosine(response.text.lower().strip(), role.lower().strip()))
    stats = role_stats[list(role_stats.keys())[np.argmax(ratios)]]

    output = {}
    tol = .1
    
    for key in ["salary", "education", "experience", "num_skills", "num_responsibilities", "num_benefits"]:
        if key not in parsed_info.keys():
            output[key] = None
            
        if isinstance(parsed_info[key], numbers.Number):
            if parsed_info[key] > (1 + tol) * stats[key]:
                output[key] = "above"
            elif parsed_info[key] < (1 - tol) * stats[key]:
                output[key] = "below"
            else:
                output[key] = "average"
    
    if type(parsed_info["education"]) == str:
        if parsed_info["education"][0] == "B":
            ed = 1
        if parsed_info["education"][0] == "M":
            ed = 2
        if parsed_info["education"][0] == "P":
            ed = 3
        else:
            ed = 0
        if ed > (1 + tol) * stats["education"]:
            output["education"] = "above"
        elif ed < (1 - tol) * stats["education"]:
            output["education"] = "below"
        else:
            output["education"] = "average"

    return output

In [51]:
# using OpenAI ChatGPT
from openai import OpenAI
import numbers

client = OpenAI(api_key = "")

def compare_job_posting(job_description, client, role_stats):
    output = {}
    for key in  ["salary", "education", "experience", "num_skills", "num_responsibilities", "num_benefits", "education"]:
        output[key] = None
        
    format = '''
    {
     "role": "job role"
     "salary": 100000,
     "education": "BS",
     "experience": 5,
     "num_skills": 5,
     "num_responsibilities": 5,
     "num_benefits": 5
    }
    '''
    description = ("where role is the job position or title, "
                   "salary is the estimated salary in dollars, "
                   "ed is the required education, "
                   "exp is the desired years of experience, "
                   "num_skills is number of skills listed, "
                   "num_responsibilities is the number of responsibilities listed, "
                   "and num_benefits is the number of benefits listed.  "
                   "If something is not provided, put a 'null' instead.  "
                   "Do not include any delimiters or special characters "
                   "other than what is needed to load this JSON string into Python.")
    
    is_json = False
    attempts = 0
    while not is_json:
        prompt = "Based off this job description:\n" + job_description + \
        "\nCan you give me a JSON object in this format with this information:\n" + format + description
        response = client.chat.completions.create(
            messages = [{"role": "user", "content": prompt}],
            model = "gpt-3.5-turbo"
        )

        # make sure we don't query too many times
        attempts += 1
        if attempts >= 3:
            return output
            
        # keep looping until we get a valid JSON format
        try:
            parsed_info = json.loads(response.choices[0].message.content)
            role = parsed_info["role"]
            is_json = True
        except:
            pass

    prompt = "Here is a list of job roles: \n" + str(list(role_stats.keys())) + f"\nCan you give me only one role from that list which is most similar to '{parsed_info['role']}'?"
    response = client.chat.completions.create(
        messages = [{"role": "user", "content": prompt}],
        model = "gpt-3.5-turbo"
    )

    # every so often AI will return a role that's not actually in the list
    # if this is the case, find the most similar role in the list to the returned role using cosine similarity
    ratios = []
    for role in list(role_stats.keys()):
        ratios.append(td.cosine(response.choices[0].message.content.lower().strip(), role.lower().strip()))
    stats = role_stats[list(role_stats.keys())[np.argmax(ratios)]]

    output = {}
    tol = .1
    
    for key in ["salary", "education", "experience", "num_skills", "num_responsibilities", "num_benefits"]:
        if key not in parsed_info.keys():
            output[key] = None
            
        if isinstance(parsed_info[key], numbers.Number):
            if parsed_info[key] > (1 + tol) * stats[key]:
                output[key] = "above"
            elif parsed_info[key] < (1 - tol) * stats[key]:
                output[key] = "below"
            else:
                output[key] = "average"
    
    if type(parsed_info["education"]) == str:
        if parsed_info["education"][0] == "B":
            ed = 1
        if parsed_info["education"][0] == "M":
            ed = 2
        if parsed_info["education"][0] == "P":
            ed = 3
        else:
            ed = 0
        if ed > (1 + tol) * stats["education"]:
            output["education"] = "above"
        elif ed < (1 - tol) * stats["education"]:
            output["education"] = "below"
        else:
            output["education"] = "average"

    return output

In [ ]:
job_description = """

"""

compare_job_posting(job_description, model, role_stats)

# **Feature 2**

What skills an applicant needs for a specific job position.

Example: I am looking for a new "software engineer" position.  When I search for "software engineer," I'd like to see a list of the most common qualifications for this position, e.g., "Python proficiency, AWS CDK, and GitHub CI/CD."

Input:
```json
{
    "role": "Frontend Web Developer",
}
```

Output:
```json
{
    "skills": [
        "HTML",
        "CSS",
        "JavaScript Frontend frameworks (e.g., React, Angular)",
        "User experience (UX)",
    ],
}
```

In [4]:
def get_keywords(skills_str):
    split = re.split(r"(?=\s[A-Z])", skills_str)
    skills = [s.strip() for s in split if s.strip()]
    return [s.lower() for s in skills]

udf_keywords = udf(get_keywords, types.ArrayType(types.StringType()))

keywords = jobs.withColumn("skills_keywords", udf_keywords(col("skills")))
keywords = keywords.withColumn("keyword", explode("skills_keywords"))
keywords_count = keywords.groupBy("Role", "keyword").count()

In [5]:
window = Window.partitionBy(keywords_count["Role"]).orderBy(keywords_count["count"].desc())
# get top 5 ranked skills keywords for each role
top_skills = keywords_count.select("*", rank().over(window).alias("rank")).filter(col("rank") <= 5)
top_skills = top_skills.groupBy("Role").agg(collect_set("keyword").alias("keywords")).toPandas()

In [ ]:
top_skills.iloc[1].keywords

# **Feature 3**

Job recommendation based on user profile.

Example: I enter in a list of qualifications I have (and maybe some other information such as desired salary).  I'd like to see a list of job positions that most fit my profile.

Input:
```json
{
    "skills": [
        "HTML",
        "CSS",
        "JavaScript Frontend frameworks (e.g., React, Angular)",
        "User experience (UX)",
    ],
    "salary": 100000,
}
```

Output:
```json
{
    "roles": [
        "Frontend Web Developer",
        "User Interface Designer",
        "Backend Developer",
    ],
}
```

## Spark Method
Upon request, run a query on a reduced version of the raw dataset to find the role most fitting of the user's requirements.

In [22]:
jobs_reduced = jobs.sample(withReplacement = False, fraction = .01)

In [23]:
user_profile = {
    "skills": ["html", "java", "python", "aws"],
    "salary": 80000,
}

In [24]:
def get_avg_salary(salary_str):
    range = [int(n) for n in re.findall(r"\d+", salary_str)]
    return statistics.mean(range) * 1000

udf_avg_salary = udf(get_avg_salary, types.FloatType())

salaries = jobs_reduced.withColumn("Salary", udf_avg_salary(col("Salary Range")))
filtered = salaries.filter(salaries["Salary"] >= user_profile["salary"])

In [25]:
def get_skills(skills_str):
    split = re.split(r"(?=\s[A-Z])", skills_str)
    return split

# explode each row by individual skill
udf_skills = udf(get_skills, types.ArrayType(types.StringType()))
skills = jobs_reduced.withColumn("skills_list", udf_skills(col("skills")))
skills = skills.withColumn("skill", explode("skills_list"))

In [26]:
def skill_similarity(skill_str):
    dists = []
    for skill in user_profile["skills"]:
        # for each skill in the provided list, calculate cosine distance
        dists.append(td.cosine(skill_str.lower().strip(), skill.lower().strip()))

    # return the cosine distance of the skill in the list with the max similarity
    return td.cosine(skill_str.lower().strip(), user_profile["skills"][np.argmax(dists)].lower().strip())

udf_similarity = udf(skill_similarity, types.FloatType())
similarity = skills.withColumn("similarity", udf_similarity(col("skill")))

In [27]:
count_cond = lambda cond: F.sum(F.when(cond,  1).otherwise(0))
# count the rows with a high enough similarity
similar_count = similarity.groupBy("Role").agg(count_cond(col("similarity") > .75).alias("num_similar"))
skills_count = similarity.groupBy("Role").count()
# for each role, get the proportion of rows that had a high enough similarity
counts = similar_count.join(skills_count, "Role").withColumn("percent_similar", (col("num_similar") / col("count")))

# get top roles
sorted = counts.orderBy(counts["percent_similar"].desc())
top = sorted.limit(5).toPandas()
list(top["Role"].values)

['Server Developer',
 'Database Developer',
 'Test Automation Engineer',
 'Automation Tester',
 'Backend Web Developer']

## JSON Method
Using spark on runtime is slow.  First, generate an output file with a list of the most common skills for each role, and then load this output file upon runtime.